# Data Science Week 3 Project 

## Please Keep scrolling for more answers... They are all below each question, I did not use a different notebook for each!! Thanks!

In [57]:
!conda install -c conda-forge folium=0.5.0

Solving environment: done

# All requested packages already installed.



In [58]:
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Getting Data 

In [59]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)

from bs4 import BeautifulSoup
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])
        

cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)

In [60]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

df.loc[df['PostalCode'] == 'M5A']
df['Neighborhood'] = df['Neighborhood'].replace(' /', ',', regex=True)

df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [61]:
df.shape

(103, 3)

# Locating Neigbours on Map

In [62]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

In [63]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [64]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')
df2.loc[df2['PostalCode'] == 'M5G']
df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [65]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [66]:
CLIENT_ID = 'YOHRI5JQ3KIYWOMLHROI3S4ZUVDG0JB2SBLQWVSP0SNKUOM0' #  Foursquare ID
CLIENT_SECRET = 'ZIL0PGRZQMV45FDOVM0UZAWHTNJ0A0WCPTPDXQIERGOR4ZQD' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [67]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [68]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=YOHRI5JQ3KIYWOMLHROI3S4ZUVDG0JB2SBLQWVSP0SNKUOM0&client_secret=ZIL0PGRZQMV45FDOVM0UZAWHTNJ0A0WCPTPDXQIERGOR4ZQD&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [69]:
results = requests.get(url).json()

In [70]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [71]:
venues = results['response']['groups'][0]['items']

In [72]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues


,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c633939e1621b8d48842553-0,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",4c633939e1621b8d48842553,"6210 Finch Ave West, Store 103",CA,Toronto,Canada,at Albion Rd.,372,"[6210 Finch Ave West, Store 103 (at Albion Rd....","[{'label': 'display', 'lat': 43.74264512142215...",43.742645,-79.589643,M9V 0A1,ON,Subway,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be58dc4cf200f479154133c-1,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",4be58dc4cf200f479154133c,1530 Albion Rd,CA,Etobicoke,Canada,Albion Mall,405,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'label': 'display', 'lat': 43.741685, 'lng':...",43.741685,-79.584487,M9V 1B4,ON,Shoppers Drug Mart,0,[]
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be70e26cf200f47e334153c-2,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",4be70e26cf200f47e334153c,1530 Albion Rd.,CA,Etobicoke,Canada,at Kipling Ave. (Albion Centre),370,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'label': 'display', 'lat': 43.74120165509377...",43.741202,-79.584545,M9V 1B4,ON,Popeyes Louisiana Kitchen,0,[]
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5112b872e4b0c0a78d7bcd27-3,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",5112b872e4b0c0a78d7bcd27,1620 Albion road,CA,Toronto,Canada,Albion Road and Finch Ave,319,"[1620 Albion road (Albion Road and Finch Ave),...","[{'label': 'display', 'lat': 43.74184023292111...",43.741840,-79.590561,NaN,ON,Sunny Foodmart,0,[]
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cd4738cdfb4a1cd4337535c-4,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",4cd4738cdfb4a1cd4337535c,1530 Albion Rd,CA,Etobicoke,Canada,Near Finch Ave. W.,413,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'label': 'display', 'lat': 43.7416936, 'lng'...",43.741694,-79.584373,M9V 1B4,ON,The Beer Store,0,[]
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b04a05bf964a520c45522e3-5,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",4b04a05bf964a520c45522e3,1530 Albion Rd,CA,Etobicoke,Canada,at Finch Ave. W.,413,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'label': 'display', 'lat': 43.741696, 'lng':...",43.741696,-79.584379,M9V 1B4,ON,Sheriff's No Frills,0,[]
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d8ba6910c4e41bdaaf7667f-6,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",4d8ba6910c4e41bdaaf7667f,"1530 Albion Road, Unit T25",CA,Etobicoke,Canada,NaN,397,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'label': 'display', 'lat': 43.74156896801906...",43.741569,-79.584489,M9V 1B4,ON,Pizza Pizza,0,[]
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c1951d6834e2d7f2d3a2a80-7,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",4c1951d6834e2d7f2d3a2a80,"1530 Albion Road, Unit F-1",CA,Toronto,Canada,NaN,427,"[1530 Albion Road, Unit F-1, Toronto ON M9V 1B...","[{'label': 'display', 'lat': 43.7417571, 'lng'...",43.741757,-79.584230,M9V 1B4,ON,McDonald's,0,[]
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-54e521a9498e559c968e8083-8,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",54e521a9498e559c968e8083,NaN,CA,NaN,Canada,NaN,391,[Canada],"[{'label': 'display', 'lat': 43.742775, 'lng':...",43.742775,-79.586985,NaN,NaN,NORI SUSHI,0,[]


In [73]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Subway,Sandwich Place,43.742645,-79.589643
1,Shoppers Drug Mart,Pharmacy,43.741685,-79.584487
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
3,Sunny Foodmart,Grocery Store,43.741840,-79.590561
4,The Beer Store,Beer Store,43.741694,-79.584373


## Getting nearest values

In [80]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [81]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [82]:
print(venues.shape)
venues.head()


(2132, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,Scarborough,43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
4,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar


In [83]:
venues.groupby('Neighborhood').count()
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 278 unique categories.


In [84]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008772,...,0.000000,0.008772,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.008772
1,Downtown Toronto,0.000000,0.000819,0.000819,0.000819,0.000819,0.001638,0.001638,0.001638,0.013923,...,0.002457,0.010647,0.001638,0.003276,0.000000,0.005733,0.001638,0.001638,0.000819,0.006552
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024194,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.008065,0.000000,0.000000,0.000000,0.016129
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.012346
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.008197,0.000000,0.004098,0.000000,0.000000,0.000000,0.000000,0.000000,0.008197,...,0.000000,0.000000,0.004098,0.008197,0.000000,0.000000,0.000000,0.000000,0.004098,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,...,0.000000,0.000000,0.000000,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.018868,0.000000,0.006289,0.000000,0.006289,0.000000,0.000000,0.000000,0.012579
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000


In [88]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Café  0.05
3            Park  0.05
4  Clothing Store  0.04
----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.05
2                Hotel  0.03
3           Restaurant  0.03
4  Japanese Restaurant  0.03
----East Toronto----
                venue  freq
0         Coffee Shop  0.06
1    Greek Restaurant  0.06
2             Brewery  0.04
3                Café  0.04
4  Italian Restaurant  0.04
----East York----
          venue  freq
0   Coffee Shop  0.06
1          Bank  0.05
2  Burger Joint  0.04
3   Pizza Place  0.04
4          Park  0.04
----Etobicoke----
            venue  freq
0     Pizza Place  0.10
1  Sandwich Place  0.07
2        Pharmacy  0.06
3     Coffee Shop  0.04
4   Grocery Store  0.04
----Mississauga----
                       venue  freq
0                Coffee Shop  0.15
1                      Hotel  0.15
2      